In [ ]:
# Copyright (c) Microsoft Corporation.
# Licensed under the MIT license.

import os
import plotly
import plotly.graph_objects as go

# produce plots for ORBIT benchmark dataset, combining train/validation/test users
DATASET_DIR = "/path/to/benchmark/root"
PHASE = "benchmark"
WITH_MODES = True
PLOT_FOLDER = "../plots"

# produce plots for just <train,validation,test> users in ORBIT benchmark dataset
# DATASET_DIR = os.path.join("/path/to/benchmark/root", "<train,validation,test>")
# PHASE = "benchmark_<train,validation,test>"
# WITH_MODES = False
# PLOT_FOLDER = "../plots"

# produce plots for ORBIT unfiltered dataset, combining all users
# DATASET_DIR = "/path/to/unfiltered/root"
# PHASE = "unfiltered"
# WITH_MODES = False
# PLOT_FOLDER = "../plots"

In [ ]:
def get_videos_per_object_by_user(path):
    user2objs, user2numvids, user2vids, vid2type = {}, {}, {}, {}
    modes = ['train', 'validation', 'test'] if WITH_MODES else ['']
    for mode in modes:
        mode_dir = os.path.join(path, mode)
        for user in os.listdir(mode_dir):
            user_dir = os.path.join(mode_dir, user)
            if os.path.isdir(user_dir):
                user2objs[user] = []
                user2numvids[user] = {'clean': [], 'clutter': [], 'clutter-pan': []}
                user2vids[user] = []
                for obj in os.listdir(user_dir):
                    obj_dir = os.path.join(user_dir, obj)
                    user2objs[user].append(obj)
                    for video_type in ['clean', 'clutter', 'clutter-pan']:
                        type_dir = os.path.join(obj_dir, video_type)
                        if os.path.exists(type_dir):
                            user2numvids[user][video_type].append( len(os.listdir(type_dir)) )
                            for video in os.listdir(type_dir):
                                user2vids[user].append(video)
                                vid2type[video] = video_type
                        else:
                            user2numvids[user][video_type].append(0)

    return user2objs, user2numvids, user2vids, vid2type

os.makedirs(PLOT_FOLDER, exist_ok=True)
user2objs, user2numvids, user2vids, vid2type = get_videos_per_object_by_user(DATASET_DIR)


In [ ]:
def plot_objects_per_user_histogram(user2objs):
    x, y, user_ids = [],[],[]
    for i, k in enumerate(sorted(user2objs.items(), key=lambda x: len(x[1]), reverse=True)):
        user_id, user_objs = k
        x.append(i)
        y.append(len(user_objs))
        user_ids.append(user_id)
    fig = go.Figure(data=[go.Bar(x=x, y=y)])
    title="objects_per_user"
    layout = go.Layout(
        height=1500,
        width=3000,
        font=dict(size=40),
    	xaxis = go.layout.XAxis(
       		title = "Collector",
            tickangle = 90,
            ticks='',
            ticktext = user_ids,
            tickvals = x,
            tickfont = dict(size=40)),
        yaxis = go.layout.YAxis(
    		title = "Number of objects",
            tickfont=dict(size=40))
    )
    fig.update_layout(layout)
    plotly.offline.iplot(fig)
    fig.write_html( os.path.join(PLOT_FOLDER, "{:}_{:}.html".format(title, PHASE)) )
    fig.write_image( os.path.join(PLOT_FOLDER, "{:}_{:}.pdf".format(title, PHASE)) )

plot_objects_per_user_histogram(user2objs)


In [ ]:
def plot_videos_by_object_per_user_histogram(user2objs, user2numvids):
    x, user_ids = [],[]
    MAX_OBJS = 15
    COLORS=[(99, 110, 250, 1), (239, 85, 59, 1), (0, 204, 150, 1), (171, 99, 250, 1), (255, 161, 90, 1), (25, 211, 243, 1), (255, 102, 146, 1), (182, 232, 128, 1), (255, 151, 255, 1), (254, 203, 82, 1), (44, 160, 44, 1), (23, 190, 207, 1), (140, 86, 75, 1), (114, 183, 178, 1), (188, 189, 34, 1), (254, 203, 82, 1) ]
    num_clean_vids_per_obj = [ [] for i in range(MAX_OBJS) ]
    num_cluttered_vids_per_obj = [ [] for i in range(MAX_OBJS) ]
    trace_names = [ "obj {:}".format(i+1) for i in range(MAX_OBJS)]
    for i, k in enumerate(sorted(user2objs.items(), key=lambda x: len(x[1]), reverse=True)):
        user_id, _ = k
        x.append(i)
        num_objs = len(user2objs[user_id])
        for j in range(MAX_OBJS):
            if j < num_objs:
                num_clean_vids_per_obj[j].append( user2numvids[user_id]['clean'][j] )
                num_cluttered_vids_per_obj[j].append( user2numvids[user_id]['clutter'][j] )
        user_ids.append(user_id)
    fig = go.Figure()
    for j in range(MAX_OBJS):
        lighter_color = list(COLORS[j])
        lighter_color[3] = 0.65
        fig.add_trace(go.Bar(x=x, y=num_clean_vids_per_obj[j], name=trace_names[j] + " clean", marker_color="rgba{:}".format(tuple(lighter_color))))
        fig.add_trace(go.Bar(x=x, y=num_cluttered_vids_per_obj[j], name=trace_names[j] + " clutter", marker_color="rgba{:}".format(COLORS[j])))
    title="videos_by_object_per_user"
    layout = go.Layout(
        height=1500,
        width=3000,
        font=dict(size=40),
    	xaxis = go.layout.XAxis(
       		title = "Collector",
            tickangle = 90,
            ticks='',
            ticktext = user_ids,
            tickvals = x,
            tickfont=dict(size=40)),
        yaxis = go.layout.YAxis(
    		title = "Number of videos by object",
            tickfont=dict(size=40)),
        barmode = 'stack',
        showlegend=True,
        legend = dict(font=dict(size=28), orientation='h', yanchor="bottom", y=1, xanchor="left", x=0)
    )
    fig.update_layout(layout)
    plotly.offline.iplot(fig)
    fig.write_html( os.path.join(PLOT_FOLDER, "{:}_{:}.html".format(title, PHASE)) )
    fig.write_image( os.path.join(PLOT_FOLDER, "{:}_{:}.pdf".format(title, PHASE)) )

plot_videos_by_object_per_user_histogram(user2objs, user2numvids)